In [1]:
import pandas as pd
from anytree import Node, AnyNode, NodeMixin,RenderTree, AbstractStyle, ContStyle
from anytree.exporter import JsonExporter
from anytree.importer import JsonImporter
import anytree
import json

In [3]:
importer = JsonImporter()

with open("C:\\Users\\mikad\\PycharmProjects\\Comp_396_YouTube_Radicalization\\tree_results\\tree_json_5cIvH-iZZfA_mika_razer_blade_2018_1_reco-3_depth-5_Conservative_YouTube.txt") as f:
    data = f.read()
    root = importer.import_(data)


In [6]:
#let's do some easy analysis
creators = []

for i in anytree.LevelOrderIter(root):
    if(i.video != None):
        creators.append(i.id)
    else:
        continue


In [9]:
creators = pd.Series(creators)
print((creators.value_counts(sort=True)))

https://www.youtube.com/watch?v=KxGPe1jD-qY           6
https://www.youtube.com/watch?v=TmNSlF7lcaw           4
https://www.youtube.com/watch?v=jey_CzIOfYE           3
https://www.youtube.com/watch?v=maAepWFvb0g&t=336s    3
https://www.youtube.com/watch?v=Q-C9mPu7r_4           2
                                                     ..
https://www.youtube.com/watch?v=zqtie0-BBj4           1
https://www.youtube.com/watch?v=LnJwH_PZXnM           1
https://www.youtube.com/watch?v=hKUffHXOb8U           1
https://www.youtube.com/watch?v=MfTv8dwGpwI           1
https://www.youtube.com/watch?v=MNjYSns0op0           1
Length: 74, dtype: int64


In [22]:
for i, v in zip((creators.value_counts(sort=True)).index, (creators.value_counts(sort=True)).values):
    if v > 1:
        res = anytree.search.findall(root, filter_=lambda node: node.id == i)
        for r in res:
            print([(an.title, an.depth) for an in r.ancestors])
        print('--------------')

[('FEMINISM VS LOGIC (OWNED COMPILATION) #2', 0), ('Jordan Peterson on Gender, Patriarchy and the Slide Towards Tyranny', 1), ('Dr. Jordan B Peterson on Femsplainers', 2), ('LEADERSHIP LAB: The Craft of Writing Effectively', 3)]
[('FEMINISM VS LOGIC (OWNED COMPILATION) #2', 0), ('Jordan Peterson on Gender, Patriarchy and the Slide Towards Tyranny', 1), ('Jordan B Peterson on masculinity and the plight of young men | BBC Sounds', 2), ('Jordan Peterson: From the Barricades of the Culture Wars', 3)]
[('FEMINISM VS LOGIC (OWNED COMPILATION) #2', 0), ('Jordan Peterson on Gender, Patriarchy and the Slide Towards Tyranny', 1), ('Jordan B Peterson on masculinity and the plight of young men | BBC Sounds', 2), ('Jordan Peterson Destroys Q&A | 25 February 2019', 3)]
[('FEMINISM VS LOGIC (OWNED COMPILATION) #2', 0), ('Jordan Peterson debate on the gender pay gap, campus protests and postmodernism', 1), ('Jordan B. Peterson on 12 Rules for Life', 2), ("Jordan Peterson's Life Advice Will Change Your